In [3]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
import scipy
import scipy.io
import matplotlib.pyplot as plt
import random
import pickle
from torch.utils.data import Dataset, DataLoader
import scipy.signal as sig
from scipy.stats import pearsonr
from utils import *
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

In [4]:
raw = scipy.io.loadmat('./datasets/raw_training_data.mat')
data_glove_1 = raw['train_dg'][0][0]
data_glove_1 = np.delete(data_glove_1, 3, 1)
data_glove_2 = raw['train_dg'][1][0]
data_glove_2 = np.delete(data_glove_2, 3, 1)
data_glove_3 = raw['train_dg'][2][0]
data_glove_3 = np.delete(data_glove_3, 3, 1)

ecog_1 = raw['train_ecog'][0][0]
ecog_2 = raw['train_ecog'][1][0]
ecog_3 = raw['train_ecog'][2][0]

labels_1 = np.argmax(data_glove_1, axis=1)
labels_2 = np.argmax(data_glove_2, axis=1)
labels_3 = np.argmax(data_glove_3, axis=1)

In [14]:
ecog_1 = filter_data(ecog_1)
ecog_2 = filter_data(ecog_2)
ecog_3 = filter_data(ecog_3)

train_test_ratio = 0.7

ecog_1_train = ecog_1[:int(train_test_ratio * ecog_1.shape[0])]
ecog_1_test = ecog_1[int(train_test_ratio * ecog_1.shape[0]):]
data_glove_1_train = data_glove_1[:int(train_test_ratio * data_glove_1.shape[0])]
data_glove_1_test = data_glove_1[int(train_test_ratio * data_glove_1.shape[0]):]

ecog_2_train = ecog_2[:int(train_test_ratio * ecog_2.shape[0])]
ecog_2_test = ecog_2[int(train_test_ratio * ecog_2.shape[0]):]
data_glove_2_train = data_glove_2[:int(train_test_ratio * data_glove_2.shape[0])]
data_glove_2_test = data_glove_2[int(train_test_ratio * data_glove_2.shape[0]):]

ecog_3_train = ecog_3[:int(train_test_ratio * ecog_3.shape[0])]
ecog_3_test = ecog_3[int(train_test_ratio * ecog_3.shape[0]):]
data_glove_3_train = data_glove_3[:int(train_test_ratio * data_glove_3.shape[0])]
data_glove_3_test = data_glove_3[int(train_test_ratio * data_glove_3.shape[0]):]

In [6]:
window = 200
X_train = ecog_1_train.reshape(-1, window, ecog_1_train.shape[1])
y_train = data_glove_1_train.reshape(-1, window, data_glove_1_train.shape[1]).mean(axis=1)
X_valid = ecog_1_test.reshape(-1, window, ecog_1_test.shape[1])
y_valid = data_glove_1_test.reshape(-1, window, data_glove_1_test.shape[1]).mean(axis=1)

In [8]:
X_train.shape

(1050, 200, 62)

In [7]:
def RiemannFeatures(X):
    Covariance = Covariances('oas')
    covar = Covariance.fit_transform(np.transpose(X, (0, 2, 1)))
    ts = TangentSpace()
    tsfeat = ts.fit_transform(covar)
    return tsfeat

In [68]:
def compute_features(X):
    # Compute Spectrum features
    feat_LL = LineLength(X)
    feat_Area = Area(X)
    feat_Energy = Energy(X)
    feat_ZCM = ZeroCrossingMean(X)
    feat_TimeAvg = averageTimeDomain(X)
    feat_Riemann = RiemannFeatures(X)
    return np.hstack([feat_Area, feat_Energy,
                      feat_TimeAvg,
                      feat_Riemann,
                      BandPower(X, 1000, 5, 15),
                      BandPower(X, 1000, 20, 25),
                      BandPower(X, 1000, 75, 115),
                      BandPower(X, 1000, 125, 160),
                      BandPower(X, 1000, 160, 175)])

In [ ]:
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace
# temp = np.transpose(X_train, (0, 2, 1))
Covariance = Covariances('oas')
covar_train = Covariance.fit_transform(np.transpose(X_train, (0, 2, 1)))
covar_valid = Covariance.transform(np.transpose(X_valid, (0, 2, 1)))
ts = TangentSpace()
tsfeat_train = ts.fit_transform(covar_train)
tsfeat_valid = ts.transform(covar_valid)

In [10]:
def concatenate(feats, overlap):
    if len(feats.shape) == 2:
        new_features = np.zeros((feats.shape[0], feats.shape[1] + 2 * overlap))
    elif len(feats.shape) == 3:
        new_features = np.zeros((feats.shape[0], feats.shape[1] + 2 * overlap, feats.shape[2]))

    for i in range(0, feats.shape[0]):
        if i > 0:
            new_features[i, 0: overlap] = feats[i - 1,-overlap:]
        new_features[i, overlap: overlap + feats.shape[1]] = feats[i]
        if i < feats.shape[0] - 1:
            new_features[i, overlap + feats.shape[1]:] = feats[i + 1, :overlap]
    return new_features

In [79]:
features_train = compute_features(X_train)
features_valid = compute_features(X_valid)

new_features_train = concatenate(features_train, 40)
new_features_valid = concatenate(features_valid, 40)

In [75]:
from xgboost import XGBRegressor
# create model instance
xgb_reg = XGBRegressor(n_estimators=500, max_depth=5, eta=0.01, subsample=0.7, colsample_bytree=0.8)
# fit model
xgb_reg.fit(new_features_train, y_train)
# make predictions
prediction_XGB = xgb_reg.predict(new_features_valid)

In [11]:
concatenate(X_train, 40).shape

(1050, 280, 62)

In [ ]:
class FingerFeatureDataset(Dataset):
    def __init__(self, ecog, dg, window=2000):
        self.ecog = np.float32(ecog.reshape(ecog.shape[0], 1, -1))
        self.ecog = (self.ecog - self.ecog.mean()) / self.ecog.std()
        self.dg = np.float32(dg)
        
    

    def __len__(self):
        return len(self.ecog)

    def __getitem__(self, idx):

        return self.ecog[idx], self.dg[idx]

In [45]:
class FingerECOGDataset(Dataset):
    def __init__(self, ecog, dg, window=200, overlap=100):
        self.ecog = np.float32(ecog.reshape(-1, 1, window,ecog.shape[1]))
        self.ecog = (self.ecog - self.ecog.mean()) / self.ecog.std()
        self.ecog = np.float32(self._concatenate(self.ecog, overlap))
        self.dg = np.float32(dg)
        
    def _concatenate(self, feats, overlap):
        new_features = np.zeros((feats.shape[0], feats.shape[1], feats.shape[2] + 2 * overlap, feats.shape[3]))
        for i in range(0, feats.shape[0]):
            if i > 0:
                new_features[i, 0, 0: overlap] = feats[i - 1, 0,-overlap:]
            new_features[i, 0, overlap: overlap + feats.shape[2]] = feats[i, 0]
            if i < feats.shape[0] - 1:
                new_features[i, 0, overlap + feats.shape[2]:] = feats[i + 1, 0, :overlap]
        return new_features

    def __len__(self):
        return len(self.ecog)

    def __getitem__(self, idx):
        
        return self.ecog[idx], self.dg[idx]

In [46]:
dataset_s1_train = FingerECOGDataset(ecog_1_train.copy(), data_glove_1_train.copy())
dataset_s1_valid = FingerECOGDataset(ecog_1_test.copy(), data_glove_1_test.copy())

train_loader = DataLoader(dataset_s1_train, batch_size=16, shuffle=False)
test_loader = DataLoader(dataset_s1_valid, batch_size=16, shuffle=False)

In [121]:
from torch import nn
class FingerFlexionCNN(nn.Module):
    def __init__(self, num_fingers) -> None:
        super(FingerFlexionCNN, self).__init__()
        
        self.conv1 = nn.Conv1d(1, 16, 5)
        self.pooling = nn.MaxPool1d(4, stride=2)
        self.bn1 = nn.BatchNorm1d(16)
        self.dropout1 = nn.Dropout(0.15)
        
        self.conv2 = nn.Conv1d(16, 32, 5)
        self.bn2 = nn.BatchNorm1d(32)
        self.dropout2 = nn.Dropout(0.25)
        
        self.conv3 = nn.Conv1d(32, 64, 5)
        self.bn3 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.25)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(19840, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_fingers)
        
        self.relu = nn.ReLU()
        
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.bn1(x)
        x = self.pooling(x)
        x = self.dropout1(x)
        
        x = self.relu(self.conv2(x))
        x = self.bn2(x)
        x = self.pooling(x)
        x = self.dropout2(x)
        
        x = self.relu(self.conv3(x))
        x = self.bn3(x)
        x = self.pooling(x)
        x = self.dropout3(x)
        
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        output = self.fc3(x)
        
        return output
    

In [58]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1,62), padding = 0)
        self.batchnorm1 = nn.BatchNorm2d(16)
        
        # Layer 2
        self.padding1 = nn.ZeroPad2d((16, 17, 0, 1))
        #self.conv2 = nn.Conv2d(1, 4, (2, 32))
        self.conv2 = nn.Conv2d(1, 4, (8, 8))
        self.batchnorm2 = nn.BatchNorm2d(4)
        self.pooling2 = nn.MaxPool2d(2, 4)
        
        # Layer 3
        self.padding2 = nn.ZeroPad2d((2, 1, 4, 3))
        self.conv3 = nn.Conv2d(4, 4, (8, 4))
        self.batchnorm3 = nn.BatchNorm2d(4)
        self.pooling3 = nn.MaxPool2d((2, 4))
        
        # FC Layer
        # NOTE: This dimension will depend on the number of timestamps per sample in your data.
        # I have 144 timepoints. 
        #4, 2, 487
        self.fc1 = nn.Linear(104, 64) #4,1,10
        self.fc2 = nn.Linear(64, 4) #4,1,10
        

    def forward(self, x):
        # Layer 1
      #  x = x.double()
        x = F.elu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.5)
        x = x.permute(0, 3, 1, 2)
        
        # Layer 2
        x = self.padding1(x)
        x = F.elu(self.conv2(x))
        x = self.batchnorm2(x)
        x = F.dropout(x, 0.5)
        x = self.pooling2(x)
        
        # Layer 3
        x = self.padding2(x)
        x = F.elu(self.conv3(x))
        x = self.batchnorm3(x)
        x = F.dropout(x, 0.25)
        x = self.pooling3(x)
        # print (x.size())
        # FC Layer
        x = torch.flatten(x, start_dim=1)# 4*2*9) # for T=128
        #x = x.view(-1,4 *2 * 9)# 4*2*9) # for T=128
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x




In [49]:
from torch.nn.modules.module import _addindent


class Conv2dWithConstraint(nn.Conv2d):
    def __init__(self, *args, max_norm=1, **kwargs):
        self.max_norm = max_norm
        super(Conv2dWithConstraint, self).__init__(*args, **kwargs)

    def forward(self, x):
        self.weight.data = torch.renorm(
            self.weight.data, p=2, dim=0, maxnorm=self.max_norm
        )
        return super(Conv2dWithConstraint, self).forward(x)


class EEGNet(nn.Module):
    def InitialBlocks(self, dropoutRate, *args, **kwargs):
        block1 = nn.Sequential(
            nn.Conv2d(1, self.F1, (1, self.kernelLength), stride=1, padding=(0, self.kernelLength // 2), bias=False),
            nn.BatchNorm2d(self.F1, momentum=0.01, affine=True, eps=1e-3),

            # DepthwiseConv2D =======================
            Conv2dWithConstraint(self.F1, self.F1 * self.D, (self.channels, 1), max_norm=1, stride=1, padding=(0, 0),
                                 groups=self.F1, bias=False),
            # ========================================

            nn.BatchNorm2d(self.F1 * self.D, momentum=0.01, affine=True, eps=1e-3),
            nn.ELU(),
            nn.AvgPool2d((1, 4), stride=4),
            nn.Dropout(p=dropoutRate))
        block2 = nn.Sequential(
            # SeparableConv2D =======================
            nn.Conv2d(self.F1 * self.D, self.F1 * self.D, (1, self.kernelLength2), stride=1,
                      padding=(0, self.kernelLength2 // 2), bias=False, groups=self.F1 * self.D),
            nn.Conv2d(self.F1 * self.D, self.F2, 1, padding=(0, 0), groups=1, bias=False, stride=1),
            # ========================================

            nn.BatchNorm2d(self.F2, momentum=0.01, affine=True, eps=1e-3),
            nn.ELU(),
            nn.AvgPool2d((1, 8), stride=8),
            nn.Dropout(p=dropoutRate))
        return nn.Sequential(block1, block2)


    def ClassifierBlock(self, inputSize, n_classes):
        return nn.Sequential(
            nn.Linear(352, n_classes, bias=False),
            nn.Softmax(dim=1))

    def CalculateOutSize(self, model, channels, samples):
        '''
        Calculate the output based on input size.
        model is from nn.Module and inputSize is a array.
        '''
        data = torch.rand(1, 1, channels, samples)
        model.eval()
        out = model(data).shape
        return out[2:]

    def __init__(self, n_classes=4, channels=60, samples=151,
                 dropoutRate=0.5, kernelLength=64, kernelLength2=16, F1=8,
                 D=2, F2=16):
        super(EEGNet, self).__init__()
        self.F1 = F1
        self.F2 = F2
        self.D = D
        self.samples = samples
        self.n_classes = n_classes
        self.channels = channels
        self.kernelLength = kernelLength
        self.kernelLength2 = kernelLength2
        self.dropoutRate = dropoutRate

        self.blocks = self.InitialBlocks(dropoutRate)
        self.blockOutputSize = self.CalculateOutSize(self.blocks, channels, samples)
        self.classifierBlock = self.ClassifierBlock(self.F2 * self.blockOutputSize[1], n_classes)

    def forward(self, x):
        x = self.blocks(x)
        x = x.view(x.size()[0], -1)  # Flatten
        x = self.classifierBlock(x)

        return x

def categorical_cross_entropy(y_pred, y_true):
    # y_pred = y_pred.cuda()
    # y_true = y_true.cuda()
    y_pred = torch.clamp(y_pred, 1e-9, 1 - 1e-9)
    return -(y_true * torch.log(y_pred)).sum(dim=1).mean()

def torch_summarize(model, show_weights=True, show_parameters=True):
    """Summarizes torch model by showing trainable parameters and weights."""
    tmpstr = model.__class__.__name__ + ' (\n'
    for key, module in model._modules.items():
        # if it contains layers let call it recursively to get params and weights
        if type(module) in [
            torch.nn.modules.container.Container,
            torch.nn.modules.container.Sequential
        ]:
            modstr = torch_summarize(module)
        else:
            modstr = module.__repr__()
        modstr = _addindent(modstr, 2)

        params = sum([np.prod(p.size()) for p in module.parameters()])
        weights = tuple([tuple(p.size()) for p in module.parameters()])

        tmpstr += '  (' + key + '): ' + modstr
        if show_weights:
            tmpstr += ', weights={}'.format(weights)
        if show_parameters:
            tmpstr +=  ', parameters={}'.format(params)
        tmpstr += '\n'

    tmpstr = tmpstr + ')'
    return tmpstr

In [59]:
net = EEGNet() #.cuda(0)
#print (net.forward(Variable(torch.Tensor(np.random.rand(1, 1, 120, 64)))))#.cuda(0))))
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.01)

net.train()
for epoch in range(50):  # loop over the dataset multiple times
    running_loss = 0.0
    correct = 0
    total = 0
    pred = []
    for (i, (ecog, dg)) in enumerate(train_loader):
        # print(ecog.shape)
        output = net(ecog)
        pred += [output.detach().numpy()]
        loss = criterion(output, dg)
        # print (loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    pred = np.concatenate(pred)
    train_cor = pearsonr(np.repeat(pred, 200, axis=0)[:,0], data_glove_1_train[:,0]).statistic
    train_loss = running_loss / len(train_loader)
    # train_acc = correct / total

    with torch.no_grad():
        running_loss = 0.0
        correct = 0
        total = 0
        pred = []
        for (i, (ecog, dg)) in enumerate(test_loader):
            output = net(ecog)
            pred += [output.detach().numpy()]
            loss = criterion(output, dg)
            running_loss += loss.item()

        pred = np.concatenate(pred)
        val_cor = pearsonr(np.repeat(pred, 200, axis=0)[:,0], data_glove_1_test[:,0]).statistic
        valid_loss = running_loss / len(test_loader)
    # print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Acc: {train_acc:.3f} | Valid loss: {valid_loss:.3f} | Valid Acc: {val_cor}')
    print(f'Epoch {epoch + 1} | Train loss: {train_loss:.3f} | Train Cor: {train_cor:.3f} | Valid loss: {valid_loss:.3f} | Valid Cor: {val_cor}')
    # break

Epoch 1 | Train loss: 0.586 | Train Cor: -0.036 | Valid loss: 0.212 | Valid Cor: -0.07221624936576038
Epoch 2 | Train loss: 0.565 | Train Cor: -0.014 | Valid loss: 0.161 | Valid Cor: -0.05836394604397254
Epoch 3 | Train loss: 0.501 | Train Cor: -0.054 | Valid loss: 0.165 | Valid Cor: -0.054167661383336216
Epoch 4 | Train loss: 0.503 | Train Cor: -0.051 | Valid loss: 0.166 | Valid Cor: -0.057242378916905944
Epoch 5 | Train loss: 0.500 | Train Cor: 0.004 | Valid loss: 0.163 | Valid Cor: -0.04840970935414928
Epoch 6 | Train loss: 0.495 | Train Cor: -0.022 | Valid loss: 0.161 | Valid Cor: -0.11725007128235547
Epoch 7 | Train loss: 0.491 | Train Cor: -0.054 | Valid loss: 0.160 | Valid Cor: -0.06002025669348629
Epoch 8 | Train loss: 0.490 | Train Cor: -0.068 | Valid loss: 0.160 | Valid Cor: -0.035377899844206015
Epoch 9 | Train loss: 0.485 | Train Cor: -0.031 | Valid loss: 0.159 | Valid Cor: -0.08681183554055165
Epoch 10 | Train loss: 0.482 | Train Cor: -0.042 | Valid loss: 0.158 | Valid Cor

KeyboardInterrupt: 

In [ ]:
torch.max(output.data, 1)[1] == dg


tensor([ True, False, False, False, False, False, False,  True, False, False,
        False, False, False, False,  True, False])

In [ ]:
pred.shape

(180, 4)

In [ ]:
np.repeat(pred, 500, axis=0).shape

(90000, 4)

In [ ]:
pearsonr(np.repeat(pred, 500, axis=0)[:,0], data_glove_1_test[:,0]).statistic

-0.0491939338496332